https://github.com/atsuch/ginnipi_dwiqc/blob/master/workflows/dwi_qc.py  
https://github.com/kaitj/mrtpipelines  
https://www.ncbi.nlm.nih.gov/pubmed/26520775

In [1]:
import os

from bids import BIDSLayout
from nipype.pipeline import engine as pe
from nipype.interfaces import utility as niu

In [2]:
def init_mrtrix_wf():
    wf = pe.Workflow(name="mrtrix_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file", "bvec_file", "bval_file"]), name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    denoise = pe.Node(DWIDenoise(), name="denoise")
    
    unring = pe.Node(MRDeGibbs(), name="unring")
    
    biascorr = pe.Node(DWIBiasCorrect(use_ants=True), name="biascorr")
    
    wf.connect([(inputnode, denoise, [("dwi_file", "in_file")]),
                (denoise, unring, [("out_file", "in_file")]),
                (unring, biascorr, [("out_file", "in_file")]),
                (inputnode, biascorr, [("bvec_file", "in_bvec"),
                                       ("bval_file", "in_bval")]),
                (biascorr, outputnode, [("out_file", "out_file")])
               ])
    
    return wf

In [11]:
def init_mrtrix_wf():
    wf = pe.Workflow(name="mrtrix_wf")
    
    inputnode = pe.Node(niu.IdentityInterface(fields=["dwi_file", "bvec_file", "bval_file"]), name="inputnode")
    
    outputnode = pe.Node(niu.IdentityInterface(fields=["out_file"]), name="outputnode")
    
    denoise = pe.Node(DWIDenoise(), name="denoise")
    
    unring = pe.Node(MRDeGibbs(), name="unring")
    
    biascorr = pe.Node(DWIBiasCorrect(use_fsl=True), name="biascorr")
    
    wf.connect([(inputnode, denoise, [("dwi_file", "in_file")]),
                (denoise, unring, [("out_file", "in_file")]),
                (unring, biascorr, [("out_file", "in_file")]),
                (inputnode, biascorr, [("bvec_file", "in_bvec"),
                                       ("bval_file", "in_bval")]),
                (biascorr, outputnode, [("out_file", "out_file")])
               ])
    
    return wf

In [12]:
def init_test_wf(dwi_file, bvec_file, bval_file):
    
    test_wf = init_mrtrix_wf()
    test_wf.base_dir = os.getcwd()
    
    inputspec = test_wf.get_node("inputnode")
    inputspec.inputs.dwi_file = dwi_file
    inputspec.inputs.bvec_file = bvec_file
    inputspec.inputs.bval_file = bval_file
    
    test_wf.write_graph(graph2use="colored")
    test_wf.config["execution"]["remove_unnecessary_outputs"] = False
    test_wf.config["execution"]["keep_inputs"] = True
    
    test_wf.run()

In [13]:
dwi_file = "/projects/mjoseph/pipelines/testing/data/sub-CMH0171/ses-01/dwi/sub-CMH0171_ses-01_acq-singleshell60dir_dwi.nii.gz"
bvec_file = "/projects/mjoseph/pipelines/testing/data/sub-CMH0171/ses-01/dwi/sub-CMH0171_ses-01_acq-singleshell60dir_dwi.bvec"
bval_file = "/projects/mjoseph/pipelines/testing/data/sub-CMH0171/ses-01/dwi/sub-CMH0171_ses-01_acq-singleshell60dir_dwi.bval"

In [17]:
init_test_wf(dwi_file, bvec_file, bval_file)

190725-10:34:27,663 nipype.workflow INFO:
	 Generated workflow graph: /mnt/tigrlab/projects/mjoseph/pipelines/testing/bin/mrtrix_wf/graph.png (graph2use=colored, simple_form=True).
190725-10:34:27,666 nipype.workflow INFO:
	 Workflow mrtrix_wf settings: ['check', 'execution', 'logging', 'monitoring']
190725-10:34:27,696 nipype.workflow INFO:
	 Running serially.
190725-10:34:27,698 nipype.workflow INFO:
	 [Node] Setting-up "mrtrix_wf.denoise" in "/mnt/tigrlab/projects/mjoseph/pipelines/testing/bin/mrtrix_wf/denoise".
190725-10:34:27,751 nipype.workflow INFO:
	 [Node] Running "denoise" ("__main__.DWIDenoise"), a CommandLine Interface with command:
dwidenoise /projects/mjoseph/pipelines/testing/data/sub-CMH0171/ses-01/dwi/sub-CMH0171_ses-01_acq-singleshell60dir_dwi.nii.gz sub-CMH0171_ses-01_acq-singleshell60dir_dwi_denoised.nii.gz
190725-10:34:29,36 nipype.interface INFO:
dwidenoise: [100%] uncompressing image "/projects/mjoseph/pipelines/testing/data/sub-CMH0171/ses-01/dwi/sub-CMH0171_se

RuntimeError: Workflow did not execute cleanly. Check log for details

In [5]:
from __future__ import print_function, division, unicode_literals, absolute_import

import os.path as op

from nipype.interfaces.base import (
    CommandLineInputSpec,
    CommandLine,
    traits,
    TraitedSpec,
    File,
    isdefined,
    Undefined,
    InputMultiObject,
)
from nipype.interfaces.mrtrix3.base import MRTrix3BaseInputSpec, MRTrix3Base

In [6]:
class DWIDenoiseInputSpec(MRTrix3BaseInputSpec):
    in_file = File(
        exists=True, argstr="%s", position=-2, mandatory=True, desc="input DWI image"
    )
    mask = File(exists=True, argstr="-mask %s", position=1, desc="mask image")
    extent = traits.Tuple(
        (traits.Int, traits.Int, traits.Int),
        argstr="-extent %d,%d,%d",
        desc="set the window size of the denoising filter. (default = 5,5,5)",
    )
    noise = File(argstr="-noise %s", desc="the output noise map")
    out_file = File(
        argstr="%s",
        name_template="%s_denoised",
        name_source="in_file",
        keep_extension=True,
        position=-1,
        desc="the output denoised DWI image",
    )


class DWIDenoiseOutputSpec(TraitedSpec):
    noise = File(desc="the output noise map", exists=True)
    out_file = File(desc="the output denoised DWI image", exists=True)


class DWIDenoise(MRTrix3Base):
    """
    Denoise DWI data and estimate the noise level based on the optimal
    threshold for PCA.

    DWI data denoising and noise map estimation by exploiting data redundancy
    in the PCA domain using the prior knowledge that the eigenspectrum of
    random covariance matrices is described by the universal Marchenko Pastur
    distribution.

    Important note: image denoising must be performed as the first step of the
    image processing pipeline. The routine will fail if interpolation or
    smoothing has been applied to the data prior to denoising.

    Note that this function does not correct for non-Gaussian noise biases.

    For more information, see
    <https://mrtrix.readthedocs.io/en/latest/reference/commands/dwidenoise.html>

    Example
    -------

    >>> import nipype.interfaces.mrtrix3 as mrt
    >>> denoise = mrt.DWIDenoise()
    >>> denoise.inputs.in_file = 'dwi.mif'
    >>> denoise.inputs.mask = 'mask.mif'
    >>> denoise.cmdline                               # doctest: +ELLIPSIS
    'dwidenoise -mask mask.mif dwi.mif dwi_denoised.mif'
    >>> denoise.run()                                 # doctest: +SKIP
    """

    _cmd = "dwidenoise"
    input_spec = DWIDenoiseInputSpec
    output_spec = DWIDenoiseOutputSpec

    # def _list_outputs(self):
    #     outputs = self.output_spec().get()
    #     # outputs["out_file"] = op.abspath(self.inputs.out_file)
    #     if self.inputs.noise != Undefined:
    #         outputs["noise"] = op.abspath(self.inputs.noise)
    #     return outputs

In [7]:
class MRDeGibbsInputSpec(MRTrix3BaseInputSpec):
    in_file = File(
        exists=True, argstr="%s", position=-2, mandatory=True, desc="input DWI image"
    )
    axes = traits.ListInt(
        #default_value=[0, 1],
        #usedefault=True,
        sep=",",
        minlen=2,
        maxlen=2,
        argstr="-axes %s",
        desc="indicate the plane in which the data was acquired (axial = 0,1; "
        "coronal = 0,2; sagittal = 1,2",
    )
    nshifts = traits.Int(
        #default_value=20,
        #usedefault=True,
        argstr="-nshifts %d",
        desc="discretization of subpixel spacing (default = 20)",
    )
    minW = traits.Int(
        #default_value=1,
        #usedefault=True,
        argstr="-minW %d",
        desc="left border of window used for total variation (TV) computation "
        "(default = 1)",
    )
    maxW = traits.Int(
        #default_value=3,
        #usedefault=True,
        argstr="-maxW %d",
        desc="right border of window used for total variation (TV) computation "
        "(default = 3)",
    )
    out_file = File(
        name_template="%s_unr",
        name_source="in_file",
        keep_extension=True,
        argstr="%s",
        position=-1,
        desc="the output unringed DWI image",
        genfile=True,
    )


class MRDeGibbsOutputSpec(TraitedSpec):
    out_file = File(desc="the output unringed DWI image", exists=True)


class MRDeGibbs(MRTrix3Base):
    """
    Remove Gibbs ringing artifacts.

    This application attempts to remove Gibbs ringing artefacts from MRI images
    using the method of local subvoxel-shifts proposed by Kellner et al.

    This command is designed to run on data directly after it has been
    reconstructed by the scanner, before any interpolation of any kind has
    taken place. You should not run this command after any form of motion
    correction (e.g. not after dwipreproc). Similarly, if you intend running
    dwidenoise, you should run this command afterwards, since it has the
    potential to alter the noise structure, which would impact on dwidenoise's
    performance.

    Note that this method is designed to work on images acquired with full
    k-space coverage. Running this method on partial Fourier ('half-scan') data
    may lead to suboptimal and/or biased results, as noted in the original
    reference below. There is currently no means of dealing with this; users
    should exercise caution when using this method on partial Fourier data, and
    inspect its output for any obvious artefacts.

    For more information, see
    <https://mrtrix.readthedocs.io/en/latest/reference/commands/mrdegibbs.html>

    Example
    -------

    >>> import nipype.interfaces.mrtrix3 as mrt
    >>> unring = mrt.MRDeGibbs()
    >>> unring.inputs.in_file = 'dwi.mif'
    >>> unring.cmdline
    'mrdegibbs -axes 0,1 -maxW 3 -minW 1 -nshifts 20 dwi.mif dwi_unr.mif'
    >>> unring.run()                                 # doctest: +SKIP
    """

    _cmd = "mrdegibbs"
    input_spec = MRDeGibbsInputSpec
    output_spec = MRDeGibbsOutputSpec

In [15]:
class DWIBiasCorrectInputSpec(MRTrix3BaseInputSpec):
    in_file = File(
        exists=True, argstr="%s", position=-2, mandatory=True, desc="input DWI image"
    )
    in_mask = File(argstr="-mask %s", desc="input mask image for bias field estimation")
    use_ants = traits.Bool(
        argstr="-ants",
        desc="use ANTS N4 to estimate the inhomogeneity field",
        xor=["use_fsl"],
    )
    use_fsl = traits.Bool(
        argstr="-fsl",
        desc="use FSL FAST to estimate the inhomogeneity field",
        xor=["use_ants"],
        min_ver="5.0.10",
    )
    _xor_grads = ("grad_file", "grad_fsl")
    bias = File(argstr="-bias %s", desc="bias field")
    out_file = File(
        name_template="%s_biascorr",
        name_source="in_file",
        keep_extension=True,
        argstr="%s",
        position=-1,
        desc="the output bias corrected DWI image",
        genfile=True,
    )


class DWIBiasCorrectOutputSpec(TraitedSpec):
    bias = File(desc="the output bias field", exists=True)
    out_file = File(desc="the output bias corrected DWI image", exists=True)


class DWIBiasCorrect(MRTrix3Base):
    """
    Perform B1 field inhomogeneity correction for a DWI volume series.

    For more information, see
    <https://mrtrix.readthedocs.io/en/latest/reference/scripts/dwibiascorrect.html>

    Example
    -------

    >>> import nipype.interfaces.mrtrix3 as mrt
    >>> bias_correct = mrt.DWIBiasCorrect()
    >>> bias_correct.inputs.in_file = 'dwi.mif'
    >>> bias_correct.cmdline
    'dwibiascorrect -ants dwi.mif dwi_biascorr.mif'
    >>> bias_correct.run()                             # doctest: +SKIP
    """

    _cmd = "dwibiascorrect"
    input_spec = DWIBiasCorrectInputSpec
    output_spec = DWIBiasCorrectOutputSpec